In [1]:
import gc

gc.collect()

import tensorflow  as tf
import tensorflow_hub as hub
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import keras
import tensorflow_text as text

# Read Labels

In [2]:
csv_path = "slim_dataset.csv"
labels_df = pd.read_csv(csv_path)
print(labels_df)

           image_name  good  crack  corrossion
0         image_0.jpg  0.33   0.00        0.66
1         image_1.jpg  0.33   0.00        0.66
2         image_3.jpg  0.33   0.00        0.66
3         image_4.jpg  0.33   0.00        0.66
4         image_5.jpg  0.66   0.00        0.33
...               ...   ...    ...         ...
5289  2_image_993.jpg  0.33   0.66        0.00
5290  0_image_994.jpg  0.66   0.33        0.00
5291  1_image_994.jpg  0.66   0.33        0.00
5292  2_image_994.jpg  0.66   0.33        0.00
5293  2_image_998.jpg  1.00   0.00        0.00

[5294 rows x 4 columns]


# Make Predictions

In [3]:
model_path = "C:/Users/chuck/OneDrive/Desktop/Honors/models/Cell_classification/All_in_one/fine_tuned_model/"
loaded_model = tf.keras.models.load_model(model_path)

In [4]:
image_folder = "C:/Users/chuck/OneDrive/Desktop/Honors/propper_datasets/images"
column_names = labels_df.columns.tolist()
prediction_df = pd.DataFrame(columns=column_names)

In [5]:
def write_numbers(image, prediction, truth):
    # Define the font and font scale
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5

    # Define the position and color for each number label
    positions = [(10, 20), (10, 50), (10, 90)]
    colors = [(0, 255, 0), (0, 0, 255), (255, 255, 0)]

    # Loop over each number and label, and write it onto the image
    for i, number in enumerate(prediction):
        label = f"{['good', 'crack', 'corrosion'][i]}: {number:.2f} ({truth[i]:.2f})"
        cv2.putText(image, label, positions[i], font, font_scale, (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(image, label, positions[i], font, font_scale, colors[i], 1, cv2.LINE_AA)

    return image
    

In [48]:
for index, row in labels_df.iterrows():
    image_name = row["image_name"]
    image_path = image_folder + "/" + image_name

    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # resize to model input size
    img_array = np.array(img)/128 - 1     # convert to numpy array

    pred = loaded_model.predict(np.expand_dims(img_array, axis=0))[0]
    row = [image_name] + list(pred)
    prediction_df.loc[len(prediction_df)] = row
    print(f"...loading {round(100*index/len(labels_df), 2)}%...")

prediction_df.to_csv("predictions.csv")

1/1 [==============================] - 2s 2s/step
...loading 0.0%...
1/1 [==============================] - 2s 2s/step
...loading 0.02%...
1/1 [==============================] - 2s 2s/step
...loading 0.04%...
1/1 [==============================] - 2s 2s/step
...loading 0.06%...
1/1 [==============================] - 2s 2s/step
...loading 0.08%...
1/1 [==============================] - 2s 2s/step
...loading 0.09%...
1/1 [==============================] - 3s 3s/step
...loading 0.11%...
1/1 [==============================] - 2s 2s/step
...loading 0.13%...
1/1 [==============================] - 3s 3s/step
...loading 0.15%...
1/1 [==============================] - 2s 2s/step
...loading 0.17%...
1/1 [==============================] - 3s 3s/step
...loading 0.19%...
1/1 [==============================] - 2s 2s/step
...loading 0.21%...
1/1 [==============================] - 2s 2s/step
...loading 0.23%...
1/1 [==============================] - 3s 3s/step
...loading 0.25%...
1/1 [================

KeyboardInterrupt: 

# Evaluate Predictions

In [10]:
prediction_df = pd.read_csv("predictions.csv")
print(prediction_df)
def all_in_one_confusion_matrix(labels_df, prediction_df):
    grouped = labels_df.groupby('crack')['image_name'].agg(list)
    good_labels_dict = grouped.to_dict()
    good_pred_dict = {key: [] for key in good_labels_dict}

    print(good_labels_dict)

    for score in good_labels_dict:
        predictions = []
        for image_name in good_labels_dict[score]:
            row = prediction_df.loc[prediction_df['image_name'] == image_name]

            print(row)
            print(image_name)
            print(row['crack'].values)
            predictions.append(row['crack'].values[0])
        
        good_pred_dict[score] = np.mean(predictions)
    
    print(good_pred_dict)



all_in_one_confusion_matrix(labels_df, prediction_df)

            image_name      good     crack  corrossion
0          image_0.jpg  0.202203  0.000628    0.797169
1          image_1.jpg  0.333719  0.001230    0.665051
2          image_2.jpg  0.992754  0.000254    0.006992
3          image_3.jpg  0.654687  0.000882    0.344431
4          image_4.jpg  0.324398  0.006223    0.669379
...                ...       ...       ...         ...
2991  1_image_2472.jpg  0.986401  0.004887    0.008712
2992  2_image_2472.jpg  0.979384  0.014056    0.006560
2993  0_image_2473.jpg  0.961616  0.038079    0.000305
2994  1_image_2473.jpg  0.930591  0.063872    0.005536
2995  2_image_2473.jpg  0.972500  0.022782    0.004718

[2996 rows x 4 columns]
{0.0: ['image_0.jpg', 'image_1.jpg', 'image_3.jpg', 'image_4.jpg', 'image_5.jpg', 'image_8.jpg', 'image_9.jpg', 'image_14.jpg', 'image_17.jpg', 'image_18.jpg', 'image_19.jpg', 'image_20.jpg', 'image_21.jpg', 'image_22.jpg', 'image_27.jpg', 'image_29.jpg', 'image_30.jpg', 'image_32.jpg', 'image_33.jpg', 'image_34.j

IndexError: index 0 is out of bounds for axis 0 with size 0